In [1]:
import findspark
findspark.init()
import pyspark # only run after findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

## Read in Data

In [32]:
bme_file_location = "sofia_small/*bme280sof.csv"
sds_file_location = "sofia_small/*sds011sof.csv"

file_type = "csv"
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

df_bme = spark.read.format(file_type) \
    .option("inferSchema", infer_schema) \
    .option("header", first_row_is_header) \
    .option("sep", delimiter) \
    .load(bme_file_location)

df_sds = spark.read.format(file_type) \
    .option("inferSchema", infer_schema) \
    .option("header", first_row_is_header) \
    .option("sep", delimiter) \
    .load(sds_file_location)


In [33]:
from pyspark.sql.functions import year, month
from pyspark.sql.functions import to_date
from pyspark.sql.functions import to_timestamp,date_format
from pyspark.sql import functions as F
from pyspark.sql.functions import count, avg
from pyspark.sql.functions import col


df_sds_transformed = df_sds.withColumn('year',year(df_sds.timestamp))\
    .withColumn('month', month(df_sds.timestamp))\
    .withColumn("day", date_format(col("timestamp"), "d"))\
    .withColumn("ts", to_date(col("timestamp")).cast("date"))

df_sds_transformed = df_sds_transformed.groupBy("ts").agg(avg("P1"), avg("P2")).orderBy(["ts"], ascending=True)

df_sds_transformed.show()

+----------+------------------+------------------+
|        ts|           avg(P1)|           avg(P2)|
+----------+------------------+------------------+
|2017-07-01|17.764459663706905| 8.341274009698298|
|2017-07-02| 9.846284524930946| 6.325375406399083|
|2017-07-03| 20.35557791635185|17.195223293020778|
|2017-07-04| 8.984114511906204| 6.868896334621589|
|2017-07-05|10.412705222705204| 7.964031059031034|
|2017-07-06| 10.85810864999049| 8.447780535930221|
|2017-07-07| 9.614079073024804| 7.430200547526521|
|2017-07-08| 12.10184730986929| 9.885236809576535|
|2017-07-09|12.441132935466957|10.319859653725107|
|2017-07-10|14.278580865387667|12.425794746989531|
|2017-07-11|16.458481004748865|13.907630592351836|
|2017-07-12|14.077904752827688|10.800456856017346|
|2017-07-13| 11.50965046888325| 8.878007956805918|
|2017-07-14| 5.461827450735781|  3.10989585931652|
|2017-07-15|10.245437171815821| 7.799760959824183|
|2017-07-16|11.484685678666041|  9.46174505220515|
|2017-07-17| 8.730244358596998|

In [34]:
df_bme_transformed = df_bme.withColumn('year',year(df_bme.timestamp))\
    .withColumn('month', month(df_bme.timestamp))\
    .withColumn("day", date_format(col("timestamp"), "d"))\
    .withColumn("ts", to_date(col("timestamp")).cast("date"))

df_bme_transformed = df_bme_transformed.groupBy("ts").agg(avg("pressure"), avg("temperature"), avg("humidity"))\
    .orderBy(["ts"], ascending=True)

df_bme_transformed.show()

+----------+-----------------+------------------+------------------+
|        ts|    avg(pressure)|  avg(temperature)|     avg(humidity)|
+----------+-----------------+------------------+------------------+
|2017-07-01|94572.18985080464| 33.33327613327619|32.792403355736745|
|2017-07-02|94441.42854684066|28.197254514672572| 44.52180304740427|
|2017-07-03|94668.76243252479| 18.25461707200767| 78.17694325226547|
|2017-07-04|95313.96683276288| 22.32803235375923|  50.4074079911003|
|2017-07-05|95440.82530922632|23.534423652694652|44.841247660928104|
|2017-07-06|95312.02019876736|25.778363851992424| 42.49701185958226|
|2017-07-07|95248.96706425186|27.469182004089852|40.482749797878675|
|2017-07-08|95059.96317789162|  25.7144688644688| 51.47889969005336|
|2017-07-09|95089.78527820377|27.075451422027033| 49.46747614048477|
|2017-07-10| 95128.1010232264|28.758966410703227|44.910974230932034|
|2017-07-11|95059.89666140139|30.580405242122854| 41.59478715493988|
|2017-07-12|94791.26359009359| 30.

In [35]:
combined_df = df_bme_transformed.join(df_sds_transformed, on=['ts'], how='left').orderBy(["ts"], ascending=True)
combined_df.show()

+----------+-----------------+------------------+------------------+------------------+------------------+
|        ts|    avg(pressure)|  avg(temperature)|     avg(humidity)|           avg(P1)|           avg(P2)|
+----------+-----------------+------------------+------------------+------------------+------------------+
|2017-07-01|94572.18985080464| 33.33327613327619|32.792403355736745|17.764459663706905| 8.341274009698298|
|2017-07-02|94441.42854684066|28.197254514672572| 44.52180304740427| 9.846284524930946| 6.325375406399083|
|2017-07-03|94668.76243252479| 18.25461707200767| 78.17694325226547| 20.35557791635185|17.195223293020778|
|2017-07-04|95313.96683276288| 22.32803235375923|  50.4074079911003| 8.984114511906204| 6.868896334621589|
|2017-07-05|95440.82530922632|23.534423652694652|44.841247660928104|10.412705222705204| 7.964031059031034|
|2017-07-06|95312.02019876736|25.778363851992424| 42.49701185958226| 10.85810864999049| 8.447780535930221|
|2017-07-07|95248.96706425186|27.4691

## Test Linear Regression

In [36]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler

vectorAssembler = VectorAssembler(inputCols = ['avg(pressure)', 'avg(temperature)', 'avg(humidity)'], outputCol = 'features')
features_df = vectorAssembler.transform(combined_df)
features_df = features_df.select(['features', 'avg(P1)'])

lr = LinearRegression(featuresCol = 'features', labelCol='avg(P1)', maxIter=10, regParam=0.3, elasticNetParam=0.8)

# Fit the model
lrModel = lr.fit(features_df)


# Summarize the model over the training set and print out some metrics
trainingSummary = lrModel.summary
print("numIterations: %d" % trainingSummary.totalIterations)
print("objectiveHistory: %s" % str(trainingSummary.objectiveHistory))
#trainingSummary.residuals.show()
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)
features_df.describe('avg(P1)').show()

numIterations: 9
objectiveHistory: [0.5, 0.46857330098728367, 0.3658516091389527, 0.36394065583390095, 0.35833477412709724, 0.3577458483095399, 0.35763854914040893, 0.3576385069330923, 0.357638506892395]
RMSE: 25.840254
r2: 0.294767
+-------+------------------+
|summary|           avg(P1)|
+-------+------------------+
|  count|               184|
|   mean| 29.66323086015132|
| stddev|30.854169933459772|
|    min| 5.461827450735781|
|    max|160.07202801241624|
+-------+------------------+



## Window Function Work

In [37]:
from pyspark.sql.window import Window
w = Window.orderBy("ts")
df1 = combined_df.withColumn("prev_avg(P1)", F.lag("avg(P1)").over(w))
df1.drop("avg(P2)").show()

+----------+-----------------+------------------+------------------+------------------+------------------+
|        ts|    avg(pressure)|  avg(temperature)|     avg(humidity)|           avg(P1)|      prev_avg(P1)|
+----------+-----------------+------------------+------------------+------------------+------------------+
|2017-07-01|94572.18985080464| 33.33327613327619|32.792403355736745|17.764459663706905|              null|
|2017-07-02|94441.42854684066|28.197254514672572| 44.52180304740427| 9.846284524930946|17.764459663706905|
|2017-07-03|94668.76243252479| 18.25461707200767| 78.17694325226547| 20.35557791635185| 9.846284524930946|
|2017-07-04|95313.96683276288| 22.32803235375923|  50.4074079911003| 8.984114511906204| 20.35557791635185|
|2017-07-05|95440.82530922632|23.534423652694652|44.841247660928104|10.412705222705204| 8.984114511906204|
|2017-07-06|95312.02019876736|25.778363851992424| 42.49701185958226| 10.85810864999049|10.412705222705204|
|2017-07-07|95248.96706425186|27.4691

In [38]:
df_lags = combined_df.select("ts", "avg(P1)")

for i in range(1, 8): 
    df_lags = df_lags.withColumn(f"P1_lag_{i}", F.lag(F.col('avg(P1)'), i).over(w))

In [39]:
lag_feature_df = df_lags.select("P1_lag_1", "P1_lag_2", "P1_lag_3", "P1_lag_4", "P1_lag_5", "P1_lag_6", "P1_lag_7", "avg(P1)")
lag_feature_df = lag_feature_df.where(col("P1_lag_7").isNotNull())

## Lagging P1 values from the past 7 days

In [40]:
vectorAssembler = VectorAssembler(inputCols = ["P1_lag_1", "P1_lag_2", "P1_lag_3", "P1_lag_4", "P1_lag_5", "P1_lag_6", "P1_lag_7"], outputCol = 'features')
lag_df = vectorAssembler.transform(lag_feature_df)
lag_df = lag_df.select(['features', 'avg(P1)'])

lr = LinearRegression(featuresCol = 'features', labelCol='avg(P1)', maxIter=10, regParam=0.3, elasticNetParam=0.8)

# Fit the model
lrModel = lr.fit(lag_df)


# Summarize the model over the training set and print out some metrics
trainingSummary = lrModel.summary
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

RMSE: 22.740847
r2: 0.467686


## Lagging P1 values from past  7 days AND lagging temps / pressures / humidities

In [ ]:
zoo

In [ ]:
# convert them into long rows with the lag information next...
         # groupBy sensor?
         # get averages for each day for each sensor
         # take the past 7 days lag information for variables
         # https://www.slideshare.net/SparkSummit/time-series-analytics-with-spark-spark-summit-east-talk-by-simon-ouellette
        

In [12]:
# https://medium.com/@sergey.ivanchuk/practical-pyspark-window-function-examples-cb5c7e1a3c41